# Main tasks:
- Web Scraping for full contents and upvotes of six answers under 5973 answered questions in the all_link.csv dataset.

### 1. Packages

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
import pandas as pd
import pickle as cPickle
import time
import datetime

In [25]:
# Fill your account infomation
user = ''
password = ''

In [26]:
driver = webdriver.Chrome(executable_path='./chromedriver')

In [27]:
driver.get('https://www.quora.com/')

In [28]:
driver.find_element_by_xpath("//input[@placeholder='Email']").send_keys(user)
driver.find_element_by_xpath("//input[@placeholder='Password']").send_keys(password)

In [29]:
driver.find_element_by_xpath("//input[@value='Login']").click()

### 2. Extract part
- Full contents of first six answers
- Upvote for each answer

In [30]:
target=pd.read_csv("all_link.csv").Answered

In [31]:
target=target[2000:3000]

In [32]:
def single_page_data(site):
    # one quora link
    driver.get(site)
    WebDriverWait(driver, 2)
    # check whether there are more link
    
    # method 1
    #try:
    #    more_path="a.ui_qtext_more_link"
    #    WebDriverWait(driver, 2).until(expected_conditions.element_to_be_clickable((By.CSS_SELECTOR, more_path))).click()
    #except:
    #    state=1
    # method 2
    try:
        [i.click() for i in driver.find_elements_by_xpath("//a[@class='ui_qtext_more_link']")]
    except:
        state=1
    WebDriverWait(driver, 2)
        
    
    # question title
    question= driver.find_element_by_xpath("//span[@class='rendered_qtext']").text.strip()
    # view
    view = [view.text for view in driver.find_elements_by_css_selector("div.ContentFooter.AnswerFooter span span.meta_num")]
    # answers length
    cnt_answers = driver.find_element_by_xpath("//div[@class='answer_count']/span[1]").text
    # answers_xpath
    answers = [answer.text for answer in driver.find_elements_by_xpath("//div[@class='ui_qtext_expanded']/span[@class='ui_qtext_rendered_qtext']")]
    # upvote_css
    upvote = [vote.text for vote in \
          driver.find_elements_by_css_selector("div.Answer.AnswerBase div div.icon_action_bar div span a div.icon_action_bar-label span.icon_action_bar-count :nth-child(2)")]
    upvote = [None if vote=='' else vote for vote in upvote]
    return question, view, cnt_answers, answers, upvote

In [33]:
def create_DF(question, view, cnt_answers, answers, upvote):
    s=pd.Series([question]*len(answers))
    singledf=pd.DataFrame({"view":view, "answers":answers,"upvote":upvote}, index=s)
    return singledf

In [34]:
df = pd.DataFrame( columns=['answers','upvote','view'])

In [35]:
start_time = datetime.datetime.now()
for site in target:
    try:
        question, view, cnt_answers, answers, upvote=single_page_data(site)
        df=pd.concat([df, create_DF(question, view, cnt_answers, answers, upvote)])
    except:
        print("something wrong for this website: ",site)
print(datetime.datetime.now() - start_time)

something wrong for this website:  https://www.quora.com/If-I-get-Bitcoins-to-my-Coinbase-wallet-in-India-how-do-I-withdraw-money-equivalent-to-the-value-of-my-bitcoins-to-my-bank-account
something wrong for this website:  https://www.quora.com/If-someone-buys-1-000-worth-of-Bitcoin-and-sells-it-at-800-due-to-any-factor-in-whose-pocket-will-these-200-go
something wrong for this website:  https://www.quora.com/If-you-had-to-invest-in-one-or-two-crypto-currencies-besides-Bitcoin-today-which-would-they-be
something wrong for this website:  https://www.quora.com/If-you-have-2-000-to-invest-in-cryptocurrencies-where-would-you-invest-Bitcoin-Litecoin-Ethereum-etc-Is-it-worth-the-risk
something wrong for this website:  https://www.quora.com/In-2011-I-bought-1-000-Bitcoin-for-basically-a-dollar-a-piece-Should-I-hold-on-to-them-or-cash-out-now
something wrong for this website:  https://www.quora.com/In-Bitcoin-what-is-it-all-about-and-how-does-it-work
something wrong for this website:  https://

In [36]:
# know how much data you collect
len(set(df.index))

955

In [37]:
df.to_csv("crawler_2000-3000.csv")

### 3. Store results into csv files

In [38]:
data = pd.read_csv("crawler_2000-3000.csv")

In [39]:
import numpy as np
data['upvote'][data['upvote'].isnull()] = 0

In [40]:
len(data['upvote'][data['upvote']==0])

1500

In [41]:
data.to_csv("crawler_2000-3000.csv")

### 4. Concatenate results from csv files

In [87]:
data1 = pd.read_csv("quora0-500.csv")
data1.head()

,questions,answers,upvote,view
0,"100 years from now, present bitcoin holders wi...",First things first: Many Bitcoin owners choose...,19,1.8k
1,"100 years from now, present bitcoin holders wi...",That is a very interesting question and one al...,10,1.9k
2,"100 years from now, present bitcoin holders wi...","Original question: ‘100 years from now, presen...",5,6.3k
3,"100 years from now, present bitcoin holders wi...",Bitcoins can be lost or misplaced because of v...,0,162
4,"100 years from now, present bitcoin holders wi...",This would seem like one of the easier problem...,15,2.5k


In [88]:
data2 = pd.read_csv("crawler_501-1000.csv")
data2.head()

,questions,answers,upvote,view
0,Do hardforks inflate the stock of bitcoins (th...,Do hardforks inflate the stock of bitcoins (th...,0,94
1,Do hardforks inflate the stock of bitcoins (th...,That is a GREAT question. Here are some simila...,1,142
2,Do hardforks inflate the stock of bitcoins (th...,Yes and no.\nI think it does inflate the stock...,1,86
3,Do hardforks inflate the stock of bitcoins (th...,A hardfork does not change the supply of bitco...,1,52
4,Do hardforks inflate the stock of bitcoins (th...,No.\nForks introduce a new and completely sepa...,0,80


In [89]:
data3 = pd.read_csv("crawler_1000-1500.csv")
data3.head()

,questions,answers,upvote,view
0,How can I register for bitcoin? How can I star...,Get The Best Bitcoin Mining Hardware\nTo begin...,3,2.2k
1,How can I register for bitcoin? How can I star...,Bitcoin mining is the technique through which ...,5,316
2,How can I register for bitcoin? How can I star...,Registration???? We are talking about bitcoins...,2,686
3,How can I register for bitcoin? How can I star...,There is no registration. Bitcoin is a decentr...,1,1.2k
4,How can I register for bitcoin? How can I star...,"Just sneaking around :)\nI have same question,...",30,2.7k


In [90]:
data4 = pd.read_csv("crawler_1501-2000.csv")
data4.head()

,questions,answers,upvote,view
0,How far does Bitcoin need to fall for it to be...,"Historically, once reached a new major price-p...",1,280
1,How far does Bitcoin need to fall for it to be...,"If any currency have a chance to crash, you sh...",0,48
2,How far does Bitcoin need to fall for it to be...,Me personally I don’t believe Bitcoin is finis...,0,72
3,How far does Bitcoin need to fall for it to be...,"Having hit an ATH at $19,000 approx and it’s c...",0,104
4,How far does Bitcoin need to fall for it to be...,It should drop hard and ONLY with news that so...,0,546


In [91]:
data5 = pd.read_csv("crawler_2000-3000.csv")
data5.head()

,questions,answers,upvote,view
0,"If Bitcoin is a truly decentralized currency, ...",I can fully agree upon the previous answer by ...,0,34
1,"If Bitcoin is a truly decentralized currency, ...","It is a truly decentralized currency, however,...",0,72
2,If Bitcoin is a viable alternative to the exis...,The automobile was (sans kinks in the engineer...,0,65
3,If Bitcoin is compared to gold and we assess t...,It’s hard to peg the value of Ethereum to anyt...,6,2.2k
4,If Bitcoin is compared to gold and we assess t...,Your question is phrased incorrectly. Bitcoin ...,1,145


In [92]:
data6 = pd.read_csv("crawler_3001-4000.csv")
data6.head()

,questions,answers,upvote,view
0,Should I continue to invest in bitcoin?,No one knows what comes next for bitcoin? But ...,117,26.1k
1,Should I continue to invest in bitcoin?,"The previous posting from Dan Zhang, #1 Most V...",71,22.6k
2,Should I continue to invest in bitcoin?,Bitcoin is a digital currency also known as cr...,25,1.9k
3,Should I continue to invest in bitcoin?,Create Your Own Cryptocurrency || Start Your O...,0,115
4,Should I continue to invest in bitcoin?,"The experts, I mean Bitcoin millionaires, pred...",32,5.7k


In [93]:
data7 = pd.read_csv("quora4000-5000.csv")
data7.head()

,questions,answers,upvote,view
0,What is the best Bitcoin cash mixer?,There aren’t many bitcoin cash mixer services ...,1,17
1,What is the best Bitcoin cash mixer?,BCHblender always works great for me!,1,13
2,What is the best bitcoin debit card?,There are a bunch of bitcoin debit cards aroun...,224,8.4k
3,What is the best bitcoin debit card?,"I’m not able to give you a good answer, unfort...",0,169
4,What is the best bitcoin debit card?,Same answer from a different question\nIf i ma...,1,283


In [94]:
data8 = pd.read_csv("quora5000-5972.csv")
data8.head()

,questions,answers,upvote,view
0,Which cryptocurrencies do you have in your por...,The information in this post was last updated ...,138,59k
1,Which cryptocurrencies do you have in your por...,I do like ETHEREUM and a few others.\nbased on...,15,5.1k
2,Which cryptocurrencies do you have in your por...,I have diversified my portfolio by investing i...,2,1k
3,Which cryptocurrencies do you have in your por...,Bitcoin (BTC) — Bitcoin is king. The OG curren...,13,3.8k
4,Which cryptocurrencies do you have in your por...,Off lately I’ve started diversifying my portfo...,11,3.1k


In [95]:
frames = [data1,data2,data3,data4,data5,data6,data7,data8]
pd.concat(frames)

,questions,answers,upvote,view
0,"100 years from now, present bitcoin holders wi...",First things first: Many Bitcoin owners choose...,19,1.8k
1,"100 years from now, present bitcoin holders wi...",That is a very interesting question and one al...,10,1.9k
2,"100 years from now, present bitcoin holders wi...","Original question: ‘100 years from now, presen...",5,6.3k
3,"100 years from now, present bitcoin holders wi...",Bitcoins can be lost or misplaced because of v...,0,162
4,"100 years from now, present bitcoin holders wi...",This would seem like one of the easier problem...,15,2.5k
5,"100 years from now, present bitcoin holders wi...",This will be dealt with in 3 ways:\nSome bitco...,3,290
6,"$20,000 for bitcoin cloud mining, how much pro...",Do not go for Cloud Mining. As an expert answe...,0,102
7,"$20,000 for bitcoin cloud mining, how much pro...",Cloud mining is either an outright scam or unp...,2,207
8,"$20,000 for bitcoin cloud mining, how much pro...",There two answers based on a detail; are you o...,0,100
9,A few BTCs were sent to my wallet. is it possi...,Downloading the full blocks is a “pain in the ...,0,38


In [78]:
all_crawler = pd.concat(frames)
len(all_crawler)

24576

In [96]:
all_crawler.to_csv("all_crawler.csv")

### 5. Check duplicate values (answers)

In [97]:
all_crawler = pd.read_csv("all_crawler.csv")
len(all_crawler)

24576

In [142]:
unique_answers = list(set(all_crawler['answers']))
len(unique_answers)

23854

In [124]:
unique_dic = {}
for each in all_crawler['answers'].values:
    if each in unique_dic:
        unique_dic[each] = unique_dic[each] + 1
    else:
        unique_dic[each] = 0

In [134]:
dup = []
for each in unique_dic:
    if unique_dic[each] != 0:
        dup.append(each)
        print(each)
        print('\n')

Bitcoin is still struggling to achieve a legal status worldwide. But some countries positively accept bitcoin.
Some of the countries are included in this video.
Where other coins are struggling to even get recognized as a good investment, bitcoin is being considered as a probable currency.
This is the case when there is little bit acceptance from these countries. Imagine what will happen when the few more countries are added to the list. The price will sky rocket.
That's why I believe Bitcoin is still at it's initial prices, it's yet to rise.


I have tried most of the bitcoin faucets available in the internet, but many proved to be low paying and not worth the time. There are still some high paying and genuine faucets available, which is definitely worth the time investing. I am currently using the following faucets by which I am making a decent amount in cryptocurrencies. Here comes the links,
Bonus Bitcoin
Bit Fun
Moon Dogecoin
http://moonliteco.in/?ref=72c480...
Moon Bitcoin
Moon D

It is not a bad thing, but is a good thing. Regulation and consumer protection are often good, not bad, even if also sometimes imperfect, downright bad at times, or in need of reform. The “fear and panic” about this is misguided. It is neither realistic, necessary, or even wise to think that we can or even should be able to be completely free of all government regulation and oversight when it comes to matters of this kind. We are also fortunate to have a society and political process in which people can work and advocate for good and beneficial rules regarding such regulation, as well as reform when necessary. It is not good to have a complete “wild west” situation regarding these matters. Now with that said, it does however seem that great reform may be needed when it comes to regulation of commodities trading in the area of precious metals. Since cryptos are to be regulated as commodities too, aside from the “securities” aspect, the same concerns would apply.


If Bitcoin was registe

In [152]:
for i in range(len(all_crawler)):
    if all_crawler['answers'].values[i] in dup:
        print(i)
        all_crawler.drop([i])

34
131
132
135
149
158
170
171
183
242
254
308
324
341
342
343
344
345
346
401
436
533
536
586
721
749
752
768
770
812
840
945
946
1067
1090
1166
1167
1168
1169
1173
1193
1227
1281
1305
1338
1343
1375
1438
1558
1598
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1668
1669
1670
1831
1836
1840
1851
1872
1936
1953
1972
1973
1974
1975
1976
1977
1980
1981
1982
1983
1984
1985
1991
2170
2192
2274
2275
2276
2277
2278
2279
2337
2369
2529
2562
2622
2704
2705
2747
2786
2899
2910
3051
3056
3057
3063
3065
3111
3128
3150
3151
3153
3173
3177
3257
3288
3340
3341
3376
3383
3388
3389
3393
3403
3409
3412
3416
3455
3489
3564
3600
3705
3723
3726
3730
3750
3754
3787
3793
3794
3832
3860
3885
3904
3999
4016
4031
4064
4065
4082
4083
4084
4085
4086
4087
4210
4211
4212
4221
4256
4316
4323
4373
4377
4379
4388
4425
4506
4531
4532
4533
4573
4574
4576
4581
4587
4610
4615
4620
4622
4623
4635
4639
4660
4717
4731
4776
4850
4854
4865
4876
4877
4900
4943
4950
4957
5008
5010
5012
5089
5090
5113
5127
5134
5136

In [ ]:
all_crawler.drop